In [2]:
!pip install langchain-ollama langgraph
%pip install -qU duckduckgo-search langchain-community aiopyston
from pyston import PystonClient,File
import asyncio
from langchain_community.tools import DuckDuckGoSearchRun
from langgraph.prebuilt import create_react_agent
from langchain_ollama.chat_models import ChatOllama
from langgraph.graph import MessagesState

from langchain_core.messages import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    ToolMessage,
    trim_messages,
)
from langchain_core.messages.utils import count_tokens_approximately


Note: you may need to restart the kernel to use updated packages.


In [5]:
recurse = 50 # model recursion limit, update if needed.
llm = ChatOllama(model="qwen3:1.7b")
search = DuckDuckGoSearchRun()
client = PystonClient()
privateState = {}

async def runCode(code:str):
    """
    sandboxed code environment
    args:
        - code (str): python code to be run
    returns:
        - result (str): the result of the code being run
    """    
    try:
        result = await client.execute("python", [File(code)])
        return result
    except Exception as e:
        return "There was an error: "+e

agent_prompt = """
You are a helpful synthetic agent.
"""

agent = create_react_agent(
    model=llm,
    tools=[search, runCode],
    prompt=agent_prompt
)

messages = []
user_input = input("User: ")
messages.append(HumanMessage(content=user_input, additional_kwargs={}, response_metadata={}))
result = await agent.ainvoke({"messages": messages})        
for msg in result["messages"]:
    msg.pretty_print()


User:  Using the coding tool provided, please give me the first 25 fibbonaci numbers.


================================ Human Message =================================

Using the coding tool provided, please give me the first 25 fibbonaci numbers.
================================== Ai Message ==================================

<think>
Okay, the user wants the first 25 Fibonacci numbers. Let me think about how to approach this.

First, I need to generate Fibonacci numbers. The sequence starts with 0 and 1, and each subsequent number is the sum of the two preceding ones. So the sequence goes 0, 1, 1, 2, 3, 5, 8, and so on.

I should check if there's a function to generate this. The user mentioned using the coding tool, so maybe the runCode function is appropriate here. Let me write a Python script that generates the first 25 numbers.

Wait, the Fibonacci sequence can be generated with a loop. Let me outline the steps:

Initialize two variables, a and b, as 0 and 1. Then, iterate from 0 to 24 (since we need 25 numbers), appending each value to a list. Then, return the list